<a href="https://colab.research.google.com/github/sipocz/_dengue/blob/main/dengue_neural_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------------scikit import 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
#_Dengue
_project="_dengue"
_PCVERSION_=False
_GITHUBVERSION_=True
_GOOGLEVERSION_=False
if _PCVERSION_:
    basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub"

if _GOOGLEVERSION_:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    basedir="/content/drive/My Drive/001_AI"

if _GITHUBVERSION_:
    !mkdir _dengue
    files=["dengue_features_train.csv","dengue_labels_train.csv","dengue_features_test.csv","/models/'model___79_0.04485_0.02581_.hdf5'"]
    path="https://raw.githubusercontent.com/sipocz/_dengue/main/"
    basedir="./"
    storeto=basedir+_project
    for f in files:
        fname=path+f
        !rm $f
        !wget $fname
        !mv $f $storeto

rm: cannot remove 'dengue_features_train.csv': No such file or directory
--2021-08-12 10:18:47--  https://raw.githubusercontent.com/sipocz/_dengue/main/dengue_features_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287139 (280K) [text/plain]
Saving to: ‘dengue_features_train.csv’

dengue_features_tra 100%[===================>] 280.41K  --.-KB/s    in 0.004s  

2021-08-12 10:18:47 (73.5 MB/s) - ‘dengue_features_train.csv’ saved [287139/287139]

rm: cannot remove 'dengue_labels_train.csv': No such file or directory
--2021-08-12 10:18:47--  https://raw.githubusercontent.com/sipocz/_dengue/main/dengue_labels_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to

In [4]:

features_train=basedir+_project+"/"+"dengue_features_train.csv"
labels_train=basedir+_project+"/"+"dengue_labels_train.csv"
features_test=basedir+_project+"/"+"dengue_features_test.csv"


X_train=pd.read_csv(features_train)
y_train=pd.read_csv(labels_train)
X_test=pd.read_csv(features_test)


In [5]:
X_train["total_cases"]=y_train["total_cases"]

In [6]:
X_test.fillna(method="bfill", inplace=True)
X_train.fillna(method="bfill", inplace=True)

In [7]:

X_train_sj=X_train[X_train.city == "sj"]
X_train_iq=X_train[X_train.city == "iq"]


In [8]:

X_test_sj=X_test[X_test.city == "sj"]
X_test_iq=X_test[X_test.city == "iq"]

In [9]:
def build_training_data(dataset,labelset, history_size = 5, target_size = 23):
    '''
    source: https://www.mikulskibartosz.name/how-to-split-a-data-frame-into-time-series-for-lstm-deep-neural-network/
    '''
    start_index = history_size
    end_index = len(dataset) - target_size

    data = []
    labels = []

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, 1)
        #print(indices)
        row=[]
        for j in indices:
            row=row+list(dataset.iloc[j])
        data.append(row)

    #label 
    for i in range(start_index, end_index):
        indices = range(i, i+target_size, 1)
        #print(indices)
        row=[]
        for j in indices:
            row=row+list(labelset.iloc[j])
        labels.append(row)


    data = np.array(data)
    labels=np.array(labels)
    #data = data.reshape((-1, 1))
    return data, labels


In [10]:
def datapipe_Xy(Xinp,yinp):
    X=Xinp.copy(deep=True)
    y=yinp.copy(deep=True)
    prediktorok=['ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw',
       'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
    X["reanalysis_air_temp_k"]=X["reanalysis_air_temp_k"]-273.15
    X["reanalysis_avg_temp_k"]=X["reanalysis_avg_temp_k"]-273.15
    X['reanalysis_max_air_temp_k']=X['reanalysis_max_air_temp_k']-273.15
    X["reanalysis_min_air_temp_k"]=X["reanalysis_min_air_temp_k"]-273.15
    X["reanalysis_dew_point_temp_k"]=X["reanalysis_dew_point_temp_k"]-273.15
    X["ndvi_N"]=(X["ndvi_ne"]+X["ndvi_nw"])/2
    X["ndvi_S"]=(X["ndvi_se"]+X["ndvi_sw"])/2


    Xprediktor=['ndvi_N','ndvi_S',
       'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
    
    yprediktor=["total_cases"]
    
    return(X[Xprediktor],y[yprediktor])

In [11]:
#test
X2,y2=datapipe_Xy(X_train_sj,X_train_sj)

In [12]:
y2.head()

,total_cases
0,4
1,5
2,4
3,3
4,6


In [13]:
X2.head()

,ndvi_N,ndvi_S,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,0.113162,0.188050,12.42,24.422857,24.592857,19.264286,26.65,22.75,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,0.156037,0.158921,22.82,25.061429,25.292857,20.801429,27.75,23.25,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,0.102608,0.164021,34.54,25.631429,25.728571,22.284286,27.35,24.15,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,0.186850,0.231721,15.36,25.837143,26.078571,22.160000,28.25,23.85,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,0.229200,0.249270,7.52,26.368571,26.514286,22.671429,28.75,24.35,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [14]:
XSJ,ySJ=build_training_data(X2,y2,4,1)

In [15]:
len(XSJ)

931

In [16]:
ySJ[0]

array([6])

In [17]:
from keras.backend import clear_session
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Flatten,Bidirectional
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam,RMSprop
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
#import tensorflow_addons as tfa

In [48]:
clear_session() 
dropout_1_rate=0.15
input_size=len(XSJ[0]) # timesteps
output_size=len(ySJ[0]) # features
learning_rate=0.00001

inputs=Input(shape=(input_size))
o1=Dense(500,activation="relu",)(inputs)
d1=Dropout(dropout_1_rate)(o1)
o2=Dense(2000,activation="relu",)(d1)
d2=Dropout(dropout_1_rate)(o2)

#o3=Dense(1000,activation="swish",)(d2)
#d3=Dropout(dropout_1_rate)(o3)

result=Dense(output_size,activation="sigmoid",)(d2)
#dropout1=Dropout(rate=dropout_1_rate)(layer2)
#result=Dense(output_size,activation="sigmoid")(dropout1)

model = Model(inputs=inputs, outputs=result)
model.compile(optimizer=Adamax(learning_rate=learning_rate), loss="mse")  #RMSprop
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 72)]              0         
_________________________________________________________________
dense (Dense)                (None, 500)               36500     
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              1002000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2001      
Total params: 1,040,501
Trainable params: 1,040,501
Non-trainable params: 0
___________________________________________________

In [49]:

x_scaler = MinMaxScaler()
XSJ_scale = pd.DataFrame(x_scaler.fit_transform(XSJ))

y_scaler=MinMaxScaler(feature_range=(0,1))
ySJ_scale=pd.DataFrame(y_scaler.fit_transform(ySJ))

In [50]:
ySJ_scale

,0
0,0.013015
1,0.004338
2,0.008677
3,0.010846
4,0.021692
...,...
926,0.006508
927,0.008677
928,0.006508
929,0.002169


In [51]:
!rm model*

In [52]:
fname="./model__"
callbacks = [#callback_LR,
       
        ModelCheckpoint(filepath=fname+"_{epoch}"+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                        verbose=2, save_best_only=True, mode='min',)]

In [53]:
XSJ_scale

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
0,0.578732,0.506411,0.031797,0.261008,0.269185,0.339933,0.307692,0.452055,0.056091,0.318138,0.031797,0.297392,0.413953,0.359684,0.440318,0.303371,0.282051,0.052305,0.637045,0.431714,0.058423,0.362993,0.384888,0.528474,0.476923,0.520548,0.031446,0.510214,0.058423,0.473460,0.330233,0.535573,0.342175,0.561798,0.564103,0.028114,0.564377,0.444793,0.088428,0.454027,0.456907,0.710356,0.415385,0.643836,0.045749,0.734988,0.088428,0.664509,0.306977,0.535573,0.363395,0.617978,0.641026,0.135338,0.678953,0.618401,0.039324,0.486881,0.514758,0.695111,0.553846,0.602740,0.024365,0.652660,0.039324,0.641761,0.348837,0.640316,0.416446,0.741573,0.705128,0.013076
1,0.637045,0.431714,0.058423,0.362993,0.384888,0.528474,0.476923,0.520548,0.031446,0.510214,0.058423,0.473460,0.330233,0.535573,0.342175,0.561798,0.564103,0.028114,0.564377,0.444793,0.088428,0.454027,0.456907,0.710356,0.415385,0.643836,0.045749,0.734988,0.088428,0.664509,0.306977,0.535573,0.363395,0.617978,0.641026,0.135338,0.678953,0.618401,0.039324,0.486881,0.514758,0.695111,0.553846,0.602740,0.024365,0.652660,0.039324,0.641761,0.348837,0.640316,0.416446,0.741573,0.705128,0.013076,0.736552,0.663403,0.019252,0.571755,0.586777,0.757841,0.630769,0.671233,0.021385,0.658555,0.019252,0.711300,0.539535,0.843874,0.899204,0.932584,0.782051,0.018960
2,0.564377,0.444793,0.088428,0.454027,0.456907,0.710356,0.415385,0.643836,0.045749,0.734988,0.088428,0.664509,0.306977,0.535573,0.363395,0.617978,0.641026,0.135338,0.678953,0.618401,0.039324,0.486881,0.514758,0.695111,0.553846,0.602740,0.024365,0.652660,0.039324,0.641761,0.348837,0.640316,0.416446,0.741573,0.705128,0.013076,0.736552,0.663403,0.019252,0.571755,0.586777,0.757841,0.630769,0.671233,0.021385,0.658555,0.019252,0.711300,0.539535,0.843874,0.899204,0.932584,0.782051,0.018960,0.620503,0.583288,0.024526,0.589551,0.603306,0.761521,0.707692,0.753425,0.046433,0.631272,0.024526,0.711670,0.241860,0.729249,0.448276,0.865169,0.782051,0.127820
3,0.678953,0.618401,0.039324,0.486881,0.514758,0.695111,0.553846,0.602740,0.024365,0.652660,0.039324,0.641761,0.348837,0.640316,0.416446,0.741573,0.705128,0.013076,0.736552,0.663403,0.019252,0.571755,0.586777,0.757841,0.630769,0.671233,0.021385,0.658555,0.019252,0.711300,0.539535,0.843874,0.899204,0.932584,0.782051,0.018960,0.620503,0.583288,0.024526,0.589551,0.603306,0.761521,0.707692,0.753425,0.046433,0.631272,0.024526,0.711670,0.241860,0.729249,0.448276,0.865169,0.782051,0.127820,0.564706,0.556728,0.008909,0.522017,0.513577,0.763273,0.538462,0.698630,0.067660,0.732451,0.008909,0.714444,0.223256,0.632411,0.416446,0.617978,0.705128,0.097091
4,0.736552,0.663403,0.019252,0.571755,0.586777,0.757841,0.630769,0.671233,0.021385,0.658555,0.019252,0.711300,0.539535,0.843874,0.899204,0.932584,0.782051,0.018960,0.620503,0.583288,0.024526,0.589551,0.603306,0.761521,0.707692,0.753425,0.046433,0.631272,0.024526,0.711670,0.241860,0.729249,0.448276,0.865169,0.782051,0.127820,0.564706,0.556728,0.008909,0.522017,0.513577,0.763273,0.538462,0.698630,0.067660,0.732451,0.008909,0.714444,0.223256,0.632411,0.416446,0.617978,0.705128,0.097091,0.523427,0.388962,0.386892,0.583390,0.564345,0.844927,0.430769,0.794521,0.052585,0.798464,0.386892,0.810616,0.069767,0.764822,0.586207,0.808989,0.641026,0.068977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,0.265885,0.632176,0.000000,0.314397,0.321133,0.428246,0.276923,0.520548,0.014549,0.386276,0.000000,0.365452,0.172093,0.284585,0.233422,0.179775,0.500000,0.043478,0.374852,0.445983,0.000000,0.291809,0.323495,0.350797,0.323077,0.506849,0.011323,0.28

In [54]:
history = model.fit(
    x=XSJ_scale,
    y=ySJ_scale,
    validation_split=0.1,
    epochs=1800,
    batch_size=1,
    callbacks=callbacks,shuffle=False
   )

Epoch 1/1800
837/837 [==============================] - 7s 7ms/step - loss: 0.0761 - val_loss: 0.0146

Epoch 00001: val_loss improved from inf to 0.01458, saving model to ./model___1_0.04698_0.01458_.hdf5
Epoch 2/1800
837/837 [==============================] - 6s 7ms/step - loss: 0.0199 - val_loss: 0.0090

Epoch 00002: val_loss improved from 0.01458 to 0.00902, saving model to ./model___2_0.01814_0.00902_.hdf5
Epoch 3/1800
837/837 [==============================] - 6s 7ms/step - loss: 0.0182 - val_loss: 0.0079

Epoch 00003: val_loss improved from 0.00902 to 0.00789, saving model to ./model___3_0.01646_0.00789_.hdf5
Epoch 4/1800
837/837 [==============================] - 6s 7ms/step - loss: 0.0182 - val_loss: 0.0075

Epoch 00004: val_loss improved from 0.00789 to 0.00746, saving model to ./model___4_0.01609_0.00746_.hdf5
Epoch 5/1800
837/837 [==============================] - 6s 7ms/step - loss: 0.0181 - val_loss: 0.0073

Epoch 00005: val_loss improved from 0.00746 to 0.00727, saving mo

KeyboardInterrupt: ignored

In [55]:
def grafikon(fx,ind,desc1,txt1,desc2="",txt2="",desc3="",txt3="",ngraf=2,c1='rgba(35,128,132,0.8)', c2='rgba(193,99,99,0.8)',c3='rgba(193,99,0,0.8)',title=None):
    '''
    fx: dataFrame
    ind: index
    desc1:column1
    txt1: label1
    desc2:column2
    txt2: label2
    ngraf: number of graph
    c1: color1
    c2: color2
    title: graph title
    '''
    
    #x_=[i for i in range(len(y_pred))]
    if title==None:
      title=txt1+txt2
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig0 = make_subplots(rows=1, cols=1)
    
    if True:
        fig0.add_trace(
            go.Scatter(x=fx[ind], y=fx[desc1], name=txt1, line=dict(color=c1) ,showlegend=True  ),

            row=1, col=1

        )
    
    if ngraf>1:
        fig0.add_trace(
            go.Scatter(x=fx[ind], y=fx[desc2], name=txt2, line=dict(color=c2) ,showlegend=True  ),

            row=1, col=1
        )
    if ngraf>2:
        fig0.add_trace(
            go.Scatter(x=fx[ind], y=fx[desc3], name=txt3, line=dict(color=c3) ,showlegend=True  ),

            row=1, col=1
        )



    fig0.update_layout(
        title=title,
        autosize=False,
        width=1200,
        height=600,
        
        )

    fig0.show()

In [56]:
#model_loaded=tf.keras.models.load_model('model___658_0.03376_0.01999_.hdf5',)

In [57]:
model_loaded=model

In [58]:
y_pred=model_loaded.predict(XSJ_scale)

In [59]:
y_pred[1]


array([0.04812714], dtype=float32)

In [60]:
def backtest_train():
    y_orig=ySJ_scale
    output=pd.DataFrame(y_pred)
    output["a"]=y_orig
    output.columns=["pred","orig"]
    output["index"]=range(0,len(y_orig))
    grafikon(output,"index","orig","orig","pred","pred")

def backtest_test():
    y_orig=ySJ_scale
    output=pd.DataFrame(y_pred)
    output["a"]=y_orig
    output.columns=["pred","orig"]
    output["index"]=range(0,len(y_orig))
    grafikon(output,"index","orig","orig","pred","pred")

In [61]:
backtest_train()